In [1]:
%%bash
# Go to folder containing the source code
cd /app/mialsuperresolutiontoolkit/
# Install the pymialsrtk package inside the python/conda environment
python setup.py install --force

running install
running build
running build_py
running build_scripts
running install_lib
copying build/lib/pymialsrtk/info.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk
copying build/lib/pymialsrtk/interfaces/registration.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/reconstruction.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/preprocess.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/__init__.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/interfaces/utils.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrtk/interfaces
copying build/lib/pymialsrtk/__init__.py -> /opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/pymialsrt

In [2]:
import os
import json
import shutil

# Imports from nipype
from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline import Node, Workflow

# Import the implemented interface from pymialsrtk
import pymialsrtk.interfaces.preprocess as preprocess


In [3]:
# Set different variables (defined in cell 2) such that we do not have to rerun cell 2
bids_dir = os.path.join('/fetaldata')
# output_dir = os.path.join('/fetaldata','derivatives')
# mask_dir = os.path.join('/fetaldata','derivatives','manual_masks')
subject = '01'

# #Make sure we start from empty derivatives
# if os.path.exists(output_dir):
#     shutil.rmtree(output_dir)
    

# #Check if mandatory derivatives dataset_description.json exists in derivatives/mialsrtk.
# # If not, it is created before running the workflow, otherwise BIDSDataGrabber is not happy and raises an error. 

# mialsrtk_dataset_description_json = os.path.join(output_dir,'dataset_description.json')

# print('Check for {}'.format(mialsrtk_dataset_description_json))
# if not os.access(mialsrtk_dataset_description_json, os.R_OK):
#     data = {'PipelineDescription':{'Name': 'MIAL Super-Resolution ToolKit', 
#                                 'Version': 'v2.0.0-beta', 
#                                 'CodeURL': 'https://github.com/sebastientourbier/mialsuperresolutiontoolkit'
#                                   },
#             'Name': 'MIAL Super-Resolution ToolKit',
#             'BIDSVersion': '1.2.0'
#            }
#     os.makedirs(output_dir)
#     with open(mialsrtk_dataset_description_json, 'w+') as outfile:
#         json.dump(data, outfile, indent=4)
#     print('File {} was created'.format(mialsrtk_dataset_description_json))
# else:
#     print('File {} already exists'.format(mialsrtk_dataset_description_json))

In [4]:
from nipype import config, logging

In [11]:
## Node linkage
def create_workflow(bids_dir,subject):
#     wf_base_dir = os.path.join("{}".format(output_dir),"superres-mri","sub-{}".format(subject),"nipype")
    output_dir = os.path.join("{}".format(bids_dir),"derivatives","superres-mri")
    wf_base_dir = os.path.join(output_dir,"sub-{}".format(subject))

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print("Ouput directory: {}".format(wf_base_dir))

    wf = Workflow(name="srr_preproc",base_dir=wf_base_dir)
    
    srr_preproc_dir = os.path.join(wf.base_dir, wf.name )
    
    
    # Initialization
    if os.path.isfile(os.path.join(output_dir,"pypeline.log")):
        os.unlink(os.path.join(output_dir,"pypeline.log"))
#         open(os.path.join(output_dir,"pypeline.log"), 'a').close()
        

    config.update_config({'logging': {'log_directory': os.path.join(output_dir),
                              'log_to_file': True},
                          'execution': {
                            'remove_unnecessary_outputs': False,
                            'stop_on_first_crash': True,
                            'stop_on_first_rerun': False,
                            'crashfile_format': "txt",
                            'write_provenance' : False,
                            },
                          'monitoring': {
                            'enabled': True
                            }
                        })
    
    logging.update_logging(config)
    iflogger = logging.getLogger('nipype.interface')

    iflogger.info("**** Processing ****")

    bg = Node(BIDSDataGrabber(infields = ['subject']),name='bids_grabber')
    bg.inputs.base_dir = bids_dir
    bg.inputs.subject = subject
    bg.inputs.index_derivatives = True
    bg.inputs.output_query = {'T2ws': dict(suffix='T2w',datatype='anat',extensions=[".nii",".nii.gz"]),
                              'masks': dict(suffix='mask',datatype='anat',extensions=[".nii",".nii.gz"])}

        
    nlmDenoise = Node(interface=preprocess.MultipleBtkNLMDenoising(),
                      name='nlmDenoise')
    nlmDenoise.inputs.bids_dir = bids_dir
    nlmDenoise.inputs.weight = 1
    
    # Sans le mask le premier correct slice intensity...
    srtkCorrectSliceIntensity01_nlm = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),
                                           name='srtkCorrectSliceIntensity01_nlm')
    srtkCorrectSliceIntensity01_nlm.inputs.bids_dir = bids_dir

    srtkCorrectSliceIntensity01 = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),
                                       name='srtkCorrectSliceIntensity01')
    srtkCorrectSliceIntensity01.inputs.bids_dir = bids_dir

    
    
    srtkSliceBySliceN4BiasFieldCorrection = Node(interface=preprocess.MultipleMialsrtkSliceBySliceN4BiasFieldCorrection(), 
                                                 name='srtkSliceBySliceN4BiasFieldCorrection')
    srtkSliceBySliceN4BiasFieldCorrection.inputs.bids_dir = bids_dir
    
    srtkSliceBySliceCorrectBiasField = Node(interface=preprocess.MultipleMialsrtkSliceBySliceCorrectBiasField(), 
                                            name='srtkSliceBySliceCorrectBiasField')
    srtkSliceBySliceCorrectBiasField.inputs.bids_dir = bids_dir
    
    
    
    srtkCorrectSliceIntensity02_nlm = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),
                                           name='srtkCorrectSliceIntensity02_nlm')
    srtkCorrectSliceIntensity02_nlm.inputs.bids_dir = bids_dir

    srtkCorrectSliceIntensity02 = Node(interface=preprocess.MultipleMialsrtkCorrectSliceIntensity(),
                                       name='srtkCorrectSliceIntensity02')
    srtkCorrectSliceIntensity02.inputs.bids_dir = bids_dir
    
    
    
    srtkIntensityStandardization01 = Node(interface=preprocess.MialsrtkIntensityStandardization(),
                                          name='srtkIntensityStandardization01')
    srtkIntensityStandardization01.inputs.bids_dir = bids_dir
    
    srtkIntensityStandardization01_nlm = Node(interface=preprocess.MialsrtkIntensityStandardization(),
                                              name='srtkIntensityStandardization01_nlm')
    srtkIntensityStandardization01_nlm.inputs.bids_dir = bids_dir
    
    
    
    srtkHistogramNormalization = Node(interface=preprocess.MialsrtkHistogramNormalization(),
                                      name='srtkHistogramNormalization')
    srtkHistogramNormalization.inputs.bids_dir = bids_dir
    
    srtkHistogramNormalization_nlm = Node(interface=preprocess.MialsrtkHistogramNormalization(),
                                          name='srtkHistogramNormalization_nlm')  
    srtkHistogramNormalization_nlm.inputs.bids_dir = bids_dir
    
    
    
    srtkIntensityStandardization02 = Node(interface=preprocess.MialsrtkIntensityStandardization(),
                                          name='srtkIntensityStandardization02')
    srtkIntensityStandardization02.inputs.bids_dir = bids_dir
    
    srtkIntensityStandardization02_nlm = Node(interface=preprocess.MialsrtkIntensityStandardization(),
                                              name='srtkIntensityStandardization02_nlm')
    srtkIntensityStandardization02_nlm.inputs.bids_dir = bids_dir
    
    # 
    srtkMaskImage01 = Node(interface=preprocess.MultipleMialsrtkMaskImage(), 
                           name='srtkMaskImage01')
    srtkMaskImage01.inputs.bids_dir = bids_dir
    #

    srtkImageReconstruction = Node(interface=preprocess.MialsrtkImageReconstruction(),
                                   name='srtkImageReconstruction')  
    srtkImageReconstruction.inputs.bids_dir = bids_dir
    srtkImageReconstruction.inputs.in_roi = "mask"
#     srtkImageReconstruction.inputs.stacksOrder = [1,2,3]  
#     run-1 axial
# run-2 coronal
# run-3 sagittal
    
    srtkTVSuperResolution = Node(interface=preprocess.MialsrtkTVSuperResolution(),
                                 name='srtkTVSuperResolution')  
    srtkTVSuperResolution.inputs.bids_dir = bids_dir
    srtkTVSuperResolution.inputs.deblurring = True
    
    
    srtkTVSuperResolution = Node(interface=preprocess.MialsrtkTVSuperResolution(),
                                 name='srtkTVSuperResolution')  
    srtkTVSuperResolution.inputs.bids_dir = bids_dir
    srtkTVSuperResolution.inputs.deblurring = True
    

    srtkRefineHRMaskByIntersection = Node(interface=preprocess.MialsrtkRefineHRMaskByIntersection(),
                                         name='srtkRefineHRMaskByIntersection')
    srtkRefineHRMaskByIntersection.inputs.bids_dir = bids_dir
    
    srtkN4BiasFieldCorrection = Node(interface=preprocess.MialsrtkN4BiasFieldCorrection(),
                                         name='srtkN4BiasFieldCorrection')
    srtkN4BiasFieldCorrection.inputs.bids_dir = bids_dir
    
    
    srtkMaskImage02 = Node(interface=preprocess.MialsrtkMaskImage(), 
                           name='srtkMaskImage02')
    srtkMaskImage02.inputs.bids_dir = bids_dir
    
    ## 
    ##
    
    
    wf.connect(bg, "T2ws", nlmDenoise, "input_images")
    wf.connect(bg, "masks", nlmDenoise, "input_masks")
    
    wf.connect(nlmDenoise, "output_images", srtkCorrectSliceIntensity01_nlm, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity01_nlm, "input_masks")
    
    wf.connect(bg, "T2ws", srtkCorrectSliceIntensity01, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity01, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity01_nlm, "output_images", srtkSliceBySliceN4BiasFieldCorrection, "input_images")
    wf.connect(bg, "masks", srtkSliceBySliceN4BiasFieldCorrection, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity01, "output_images", srtkSliceBySliceCorrectBiasField, "input_images")
    wf.connect(srtkSliceBySliceN4BiasFieldCorrection, "output_fields", srtkSliceBySliceCorrectBiasField, "input_fields")
    wf.connect(bg, "masks", srtkSliceBySliceCorrectBiasField, "input_masks")
    
    wf.connect(srtkSliceBySliceCorrectBiasField, "output_images", srtkCorrectSliceIntensity02, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity02, "input_masks")
    
    wf.connect(srtkSliceBySliceN4BiasFieldCorrection, "output_images", srtkCorrectSliceIntensity02_nlm, "input_images")
    wf.connect(bg, "masks", srtkCorrectSliceIntensity02_nlm, "input_masks")
    
    wf.connect(srtkCorrectSliceIntensity02, "output_images", srtkIntensityStandardization01, "input_images")
    
    wf.connect(srtkCorrectSliceIntensity02_nlm, "output_images", srtkIntensityStandardization01_nlm, "input_images")
    
    wf.connect(srtkIntensityStandardization01, "output_images", srtkHistogramNormalization, "input_images")
    wf.connect(bg, "masks", srtkHistogramNormalization, "input_masks")
    
    wf.connect(srtkIntensityStandardization01_nlm, "output_images", srtkHistogramNormalization_nlm, "input_images")
    wf.connect(bg, "masks", srtkHistogramNormalization_nlm, "input_masks")
    
    wf.connect(srtkHistogramNormalization, "output_images", srtkIntensityStandardization02, "input_images")
    
    wf.connect(srtkHistogramNormalization_nlm, "output_images", srtkIntensityStandardization02_nlm, "input_images")
    
    
    wf.connect(srtkIntensityStandardization02_nlm, "output_images", srtkMaskImage01, "input_images")
    wf.connect(bg, "masks", srtkMaskImage01, "input_masks")
    
    wf.connect(srtkMaskImage01, "output_images", srtkImageReconstruction, "input_images")
    wf.connect(bg, "masks", srtkImageReconstruction, "input_masks")
    
    
    wf.connect(srtkIntensityStandardization02, "output_images", srtkTVSuperResolution, "input_images")
    wf.connect(srtkImageReconstruction, "output_transforms", srtkTVSuperResolution, "input_transforms")
    wf.connect(srtkImageReconstruction, "output_sdi", srtkTVSuperResolution, "input_sdi")
    wf.connect(bg, "masks", srtkTVSuperResolution, "input_masks")
    
    
    wf.connect(srtkIntensityStandardization02, "output_images", srtkRefineHRMaskByIntersection, "input_images")
    wf.connect(bg, "masks", srtkRefineHRMaskByIntersection, "input_masks")
    wf.connect(srtkImageReconstruction, "output_transforms", srtkRefineHRMaskByIntersection, "input_transforms")
    wf.connect(srtkTVSuperResolution, "output_sr", srtkRefineHRMaskByIntersection, "input_sr")
    
    wf.connect(srtkTVSuperResolution, "output_sr", srtkN4BiasFieldCorrection, "input_image")
    wf.connect(srtkRefineHRMaskByIntersection, "output_SRmask", srtkN4BiasFieldCorrection, "input_mask")
    
    wf.connect(srtkTVSuperResolution, "output_sr", srtkMaskImage02, "in_file")
    wf.connect(srtkRefineHRMaskByIntersection, "output_SRmask", srtkMaskImage02, "in_mask")
    
    return wf

In [12]:
m_wf = create_workflow(bids_dir, subject)
m_wf.run()
m_wf.write_graph()

Ouput directory: /fetaldata/derivatives/superres-mri/sub-01
200116-19:17:03,399 nipype.interface INFO:
	 **** Processing ****
200116-19:17:03,425 nipype.workflow INFO:
	 Workflow srr_preproc settings: ['check', 'execution', 'logging', 'monitoring']
200116-19:17:03,436 nipype.workflow INFO:
	 Running serially.
200116-19:17:03,437 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.bids_grabber" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/bids_grabber".
200116-19:17:03,439 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_preproc.bids_grabber".
200116-19:17:03,442 nipype.workflow INFO:
	 [Node] Running "bids_grabber" ("nipype.interfaces.io.BIDSDataGrabber")


/opt/conda/envs/pymialsrtk-env/lib/python3.7/site-packages/bids/layout/layout.py:659: UserWarning: In pybids 0.9.0, the 'extensions' filter was deprecated in favor of 'extension'. The former will stop working in 0.11.0.
  warnings.warn("In pybids 0.9.0, the 'extensions' filter was "


200116-19:17:04,785 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.bids_grabber".
200116-19:17:04,785 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkCorrectSliceIntensity01" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity01".
200116-19:17:04,787 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_preproc.srtkCorrectSliceIntensity01".
200116-19:17:04,789 nipype.workflow INFO:
	 [Node] Running "srtkCorrectSliceIntensity01" ("pymialsrtk.interfaces.preprocess.MultipleMialsrtkCorrectSliceIntensity")
... cmd: mialsrtkCorrectSliceIntensity "/fetaldata/sub-01/anat/sub-01_run-1_T2w.nii.gz" "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-1_T2w_desc-brain_mask.nii.gz" "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity01/sub-01_run-1_T2w_csi.nii.gz"
... cmd: mialsrtkCorrectSliceIntensity "/fetaldata/sub-01/anat/sub-01_run-2_T2w.nii.gz" "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_r

... cmd: mialsrtkSliceBySliceN4BiasFieldCorrection "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity01_nlm/sub-01_run-5_T2w_nlm_csi.nii.gz" "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-1_T2w_desc-brain_mask.nii.gz" "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkSliceBySliceN4BiasFieldCorrection/sub-01_run-5_T2w_nlm_csi_sliceN4corr.nii.gz" "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkSliceBySliceN4BiasFieldCorrection/sub-01_run-5_T2w_nlm_csi_sliceN4field.nii.gz"
... cmd: mialsrtkSliceBySliceN4BiasFieldCorrection "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity01_nlm/sub-01_run-1_T2w_nlm_csi.nii.gz" "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-2_T2w_desc-brain_mask.nii.gz" "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkSliceBySliceN4BiasFieldCorrection/sub-01_run-1_T2w_nlm_csi_sliceN4corr.nii.gz" "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/sr

200116-19:17:31,211 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.srtkIntensityStandardization01_nlm".
200116-19:17:31,212 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkHistogramNormalization_nlm" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkHistogramNormalization_nlm".
200116-19:17:31,214 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_preproc.srtkHistogramNormalization_nlm".
200116-19:17:31,217 nipype.workflow INFO:
	 [Node] Running "srtkHistogramNormalization_nlm" ("pymialsrtk.interfaces.preprocess.MialsrtkHistogramNormalization")
... cmd: python /usr/local/bin/mialsrtkHistogramNormalization.py  -i "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkIntensityStandardization01_nlm/sub-01_run-6_T2w_nlm_csi_sliceN4corr_csi_ist.nii.gz" -o "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkHistogramNormalization_nlm/sub-01_run-6_T2w_nlm_csi_sliceN4corr_csi_ist_HistNorm.nii.gz" -m "/fetaldata/derivatives/manual_masks/sub-

200116-19:17:34,223 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.srtkMaskImage01".
200116-19:17:34,224 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkImageReconstruction" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkImageReconstruction".
200116-19:17:34,226 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_preproc.srtkImageReconstruction".
200116-19:17:34,229 nipype.workflow INFO:
	 [Node] Running "srtkImageReconstruction" ("pymialsrtk.interfaces.preprocess.MialsrtkImageReconstruction")
input_image ['/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkMaskImage01/sub-01_run-4_T2w_nlm_csi_sliceN4corr_csi_ist_HistNorm_ist_masked.nii.gz', '/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkMaskImage01/sub-01_run-6_T2w_nlm_csi_sliceN4corr_csi_ist_HistNorm_ist_masked.nii.gz', '/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkMaskImage01/sub-01_run-3_T2w_nlm_csi_sliceN4corr_csi_ist_HistNorm_ist_masked.nii.gz', '/fetald

200116-19:21:33,822 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.srtkSliceBySliceCorrectBiasField".
200116-19:21:33,823 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkCorrectSliceIntensity02" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity02".
200116-19:21:33,825 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_preproc.srtkCorrectSliceIntensity02".
200116-19:21:33,827 nipype.workflow INFO:
	 [Node] Running "srtkCorrectSliceIntensity02" ("pymialsrtk.interfaces.preprocess.MultipleMialsrtkCorrectSliceIntensity")
... cmd: mialsrtkCorrectSliceIntensity "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkSliceBySliceCorrectBiasField/sub-01_run-3_T2w_csi_sliceFldCorrected.nii.gz" "/fetaldata/derivatives/manual_masks/sub-01/anat/sub-01_run-1_T2w_desc-brain_mask.nii.gz" "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkCorrectSliceIntensity02/sub-01_run-3_T2w_csi_sliceFldCorrected_csi.nii.gz"
... cmd: mialsrt

200116-19:21:37,306 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.srtkHistogramNormalization".
200116-19:21:37,307 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkIntensityStandardization02" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkIntensityStandardization02".
200116-19:21:37,309 nipype.workflow INFO:
	 [Node] Outdated cache found for "srr_preproc.srtkIntensityStandardization02".
200116-19:21:37,311 nipype.workflow INFO:
	 [Node] Running "srtkIntensityStandardization02" ("pymialsrtk.interfaces.preprocess.MialsrtkIntensityStandardization")
... cmd: mialsrtkIntensityStandardization --input "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkHistogramNormalization/sub-01_run-6_T2w_csi_sliceFldCorrected_csi_ist_HistNorm.nii.gz" --output "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkIntensityStandardization02/sub-01_run-6_T2w_csi_sliceFldCorrected_csi_ist_HistNorm_ist.nii.gz" --input "/fetaldata/derivatives/superres-mri/sub-01/sr

200116-19:28:36,680 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.srtkRefineHRMaskByIntersection".
200116-19:28:36,681 nipype.workflow INFO:
	 [Node] Setting-up "srr_preproc.srtkMaskImage02" in "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkMaskImage02".
200116-19:28:36,684 nipype.workflow INFO:
	 [Node] Running "srtkMaskImage02" ("pymialsrtk.interfaces.preprocess.MialsrtkMaskImage")
... cmd: mialsrtkMaskImage -i "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkTVSuperResolution/sub-01_run-5_T2w_csi_sliceFldCorrected_csi_ist_HistNorm_ist_SR.nii.gz" -m "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkRefineHRMaskByIntersection/sub-01_run-5_T2w_csi_sliceFldCorrected_csi_ist_HistNorm_ist_srMask.nii.gz" -o "/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/srtkMaskImage02/sub-01_run-5_T2w_csi_sliceFldCorrected_csi_ist_HistNorm_ist_SR_masked.nii.gz"
200116-19:28:37,201 nipype.workflow INFO:
	 [Node] Finished "srr_preproc.srtkMaskImage02".
20011

'/fetaldata/derivatives/superres-mri/sub-01/srr_preproc/graph.png'

In [ ]:
%%bash
nipypecli show /home/priscille/Documents/MyWork/MIAL/mialsuperresolutiontoolkit/data/derivatives/superres-mri/sub-01/srr_preproc/srtkTVSuperResolution/_node.pklz